In [ ]:
import numpy as np
import cv2
import nbimporter
import lib.util as util
from lib.hough_circles_acc import *
from lib.hough_peaks import *

In [ ]:
def find_circles(edge_img, radius_range=[1,2], threshold=100, nhood_size=10):
    n = radius_range[1] - radius_range[0]
    H_size = (n,) + edge_img.shape
    H = np.zeros(H_size, dtype=np.uint)
    centers = ()
    radii = np.arange(radius_range[0], radius_range[1])
    valid_radii = np.array([], dtype=np.uint)
    num_circles = 0
    
    for i in range(len(radii)):
        H[i] = hough_circles_acc(edge_img, radii[i])
        peaks = hough_peaks(H[i], numpeaks=10, threshold=threshold, nhood_size=nhood_size)
        if peaks.shape[0]:
            valid_radii = np.append(valid_radii, radii[i])
            centers = centers + (peaks,)
            for peak in peaks:
                cv2.circle(edge_img, tuple(peak[::-1]), radii[i]+1, (0,0,0), -1)
        num_circles += peaks.shape[0]
        print('Progress: %d%% - Circles: %d\033[F\r'%(100*i/len(radii), num_circles))
        
    centers = np.array(centers)
    return centers, valid_radii.astype(np.uint)